In [1]:
import importlib.util
import pip

def check_and_install(package):
    try:
        spec = importlib.util.find_spec(package)
        if spec is None:
            print(f"Installing {package}...")
            pip.main(['install', package])
    except Exception as e:
        print(f"Error installing {package}: {e}")

check_and_install('pyfinance')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from pyfinance.options import BSM

# Feather myös ok. Copilotin kanssa csv toimii paremmin
data = pd.read_csv("firma_data_better.csv", parse_dates=["Date"])

print(data.head())
print(data.columns)

        Date  Underlying  C445   C450   C455  C460   C465   C470   C475  \
0 2025-09-11      470.73   NaN    NaN    NaN  25.4  19.27  18.20  15.97   
1 2025-09-12      471.31   NaN  31.79  27.75  25.0  19.10    NaN  15.68   
2 2025-09-15      473.25   NaN    NaN    NaN   NaN    NaN  18.90  16.00   
3 2025-09-16      474.32   NaN    NaN    NaN   NaN  22.36  21.39  18.10   
4 2025-09-17      473.12   NaN    NaN    NaN  25.6    NaN  19.35    NaN   

    C480  ...   P475  P480  P485  P490  P495  P500  P505  P510  P515  P520  
0    NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1  14.28  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2  13.90  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3  15.70  ...  14.25   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4    NaN  ...    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 34 columns]
Index(['Date', 'Underlying', 'C445', 'C450', 'C455', 'C460', 'C4

In [2]:
# --- Find ATM call column at t0 ---

call_cols = [c for c in data.columns if c.startswith("C")]
strikes = np.array([int(c[1:]) for c in call_cols])   # e.g. "C470" -> 470

S0 = data["Underlying"].iloc[0]
idx_atm = np.argmin(np.abs(strikes - S0))
K = strikes[idx_atm]
atm_col = call_cols[idx_atm]

print(f"Initial spot S0 = {S0:.2f}")
print(f"Using ATM call column {atm_col} with strike K = {K}")

# --- Build time series for that call and underlying ---
data[atm_col] = data[atm_col].interpolate() #Fill small gaps. Maybe wanted?
df_hedge = (
    data[["Date", "Underlying", atm_col]]
    .dropna()
    .reset_index(drop=True)
    .rename(columns={"Underlying": "S", atm_col: "C"})
)

print(df_hedge.head())
print(f"Number of trading days in series: {len(df_hedge)}")

Initial spot S0 = 470.73
Using ATM call column C470 with strike K = 470
        Date       S      C
0 2025-09-11  470.73  18.20
1 2025-09-12  471.31  18.55
2 2025-09-15  473.25  18.90
3 2025-09-16  474.32  21.39
4 2025-09-17  473.12  19.35
Number of trading days in series: 37


In [3]:
n = len(df_hedge)
dt = 1.0 / 252.0  # 252 trading days in a year

# Time to maturity (in years) for each observation, last day T=0
T = (n - 1 - np.arange(n)) * dt
df_hedge["T"] = T

df_hedge.head()

,Date,S,C,T
0,2025-09-11,470.73,18.20,0.142857
1,2025-09-12,471.31,18.55,0.138889
2,2025-09-15,473.25,18.90,0.134921
3,2025-09-16,474.32,21.39,0.130952
4,2025-09-17,473.12,19.35,0.126984


In [4]:
# Risk-free rate (can be changed)
r = 0.0

S0 = df_hedge.loc[0, "S"]
C0 = df_hedge.loc[0, "C"]
T0 = df_hedge.loc[0, "T"]

print(f"T0 (years) = {T0:.4f} (~{T0*252:.1f} trading days to maturity)")

# Initial guess for sigma (doesn't matter much, implied_vol does root-finding)
sigma_guess = 0.2

op0 = BSM(kind='call', S0=S0, K=K, T=T0, r=r, sigma=sigma_guess)
sigma0 = op0.implied_vol(value=C0)

print(f"Implied volatility at t0: sigma0 = {sigma0:.4f}")

T0 (years) = 0.1429 (~36.0 trading days to maturity)
Implied volatility at t0: sigma0 = 0.2515


In [18]:
def simulate_delta_hedge(
    S, C, T, K, r, sigma0,
    rebalance_every=1,
    cost_rate=0.0,
    use_implied_vol=False,
):
    """
    Delta hedging simulation with optional time-varying implied volatility.

    Parameters
    ----------
    S, C, T : 1D numpy arrays of same length n
        S[i] : spot price at time i
        C[i] : call price at time i (market)
        T[i] : time-to-maturity in years at time i
    K, r, sigma0 :
        K      : strike
        r      : risk-free rate
        sigma0 : initial volatility (used as constant vol OR initial IV guess)
    rebalance_every : int
        1 = daily hedge, 2 = every 2 days, etc.
    cost_rate : float
        proportional cost per unit notional traded in underlying, e.g. 0.05 = 5%
    use_implied_vol : bool
        False (default): use constant sigma0 for all deltas 
        True          : at each re-hedge date, recalibrate sigma from C[i] using
                        BSM(...).implied_vol(value=C[i]) and use that for deltas
                        until the next rebalance.

    Returns
    -------
    dict with:
        deltas, A_no_cost, A_with_cost, E_no_cost, E_with_cost,
        costs, total_cost, cum_error_no_cost, cum_error_with_cost,
        sigma_used (volatility actually used at each time step)
    """

    S = np.asarray(S)
    C = np.asarray(C)
    T = np.asarray(T)
    n = len(S)

    # Storage
    deltas = np.zeros(n)
    A_no_cost = np.zeros(n - 1)
    A_with_cost = np.zeros(n - 1)
    costs = np.zeros(n - 1)        # cost incurred at start of each period i
    sigma_used = np.zeros(n)       # volatility actually used at each step

    current_delta = 0.0            # start flat in underlying (Vähän epävarma)
    sigma_current = sigma0         # start with given sigma

    for i in range(n - 1):
        # Decide whether to rebalance at start of period [i, i+1]
        if i == 0 or (i % rebalance_every == 0):

            # Optionally update sigma_current using implied vol from C[i]
            if use_implied_vol and T[i] > 0 and C[i] > 0 and S[i] > 0:
                try:
                    op_iv = BSM(
                        kind="call",
                        S0=S[i],
                        K=K,
                        T=T[i],
                        r=r,
                        sigma=max(1e-4, sigma_current)  # initial guess
                    )
                    sigma_iv = op_iv.implied_vol(value=C[i])
                    # Only accept "reasonable" numbers
                    if np.isfinite(sigma_iv) and sigma_iv > 0:
                        sigma_current = sigma_iv
                except Exception:
                    # If implied vol fails, keep previous sigma_current
                    pass

            # Compute new delta at time i using sigma_current
            if T[i] <= 0:
                # At/after maturity: delta = 1 if ITM, else 0
                new_delta = 1.0 if S[i] > K else 0.0
            else:
                op = BSM(kind="call", S0=S[i], K=K, T=T[i], r=r, sigma=sigma_current)
                new_delta = op.delta()

            traded = abs(new_delta - current_delta)
            costs[i] = cost_rate * traded * S[i]     # proportional cost
            current_delta = new_delta

        deltas[i] = current_delta
        sigma_used[i] = sigma_current

        # Changes in option and spot over [i, i+1]
        dC = C[i + 1] - C[i]
        dS = S[i + 1] - S[i]

        # Hedging error without transaction costs
        A_no_cost[i] = dC - current_delta * dS

        # Hedging error including the transaction cost at time i
        A_with_cost[i] = A_no_cost[i] - costs[i]

    # Hold the last delta and sigma constant to expiry (for information)
    deltas[-1] = current_delta
    sigma_used[-1] = sigma_current

    # Mean squared errors
    E_no_cost = np.mean(A_no_cost**2)
    E_with_cost = np.mean(A_with_cost**2)

    # Cumulative tracking error curves (for plotting P&L of hedged portfolio)
    cum_error_no_cost = np.cumsum(A_no_cost)
    cum_error_with_cost = np.cumsum(A_with_cost)

    result = {
        "deltas": deltas,
        "A_no_cost": A_no_cost,
        "A_with_cost": A_with_cost,
        "E_no_cost": E_no_cost,
        "E_with_cost": E_with_cost,
        "costs": costs,
        "total_cost": float(np.sum(costs)),
        "cum_error_no_cost": cum_error_no_cost,
        "cum_error_with_cost": cum_error_with_cost,
        "sigma_used": sigma_used,
    }
    return result


In [15]:
S = df_hedge["S"].values
C = df_hedge["C"].values
T = df_hedge["T"].values


freqs = [1, 2, 5, 10]   # hedge daily, every 2 days, 5 days, 10 days
cost_rate = 0.05        # 5% transaction cost per notional traded

results = {}

for f in freqs:
    res = simulate_delta_hedge(S, C, T, K, r, sigma0,
                               rebalance_every=f,
                               cost_rate=cost_rate,
                               use_implied_vol=True)
    results[f] = res
    print(f"=== Rebalance every {f} day(s) ===")
    print(f"  MSE without costs: {res['E_no_cost']:.6e}")
    print(f"  MSE with costs:    {res['E_with_cost']:.6e}")
    print(f"  Total transaction cost: {res['total_cost']:.4f}")
    print()


=== Rebalance every 1 day(s) ===
  MSE without costs: 1.551501e+01
  MSE with costs:    2.338582e+01
  Total transaction cost: 55.0589

=== Rebalance every 2 day(s) ===
  MSE without costs: 1.474123e+01
  MSE with costs:    2.085512e+01
  Total transaction cost: 26.8235

=== Rebalance every 5 day(s) ===
  MSE without costs: 1.368372e+01
  MSE with costs:    1.990411e+01
  Total transaction cost: 22.8861

=== Rebalance every 10 day(s) ===
  MSE without costs: 1.445444e+01
  MSE with costs:    1.997279e+01
  Total transaction cost: 20.5521



In [7]:
def call_greeks(S, K, T, r, sigma):
    """
    Return (delta, vega) for a European call using the BSM class.
    Vega here is ∂C/∂σ as in the notes (kappa).
    """
    if T <= 0 or S <= 0 or K <= 0:
        # At expiry, Delta is 1{S > K}, Vega is 0
        delta = 1.0 if S > K else 0.0
        vega = 0.0
    else:
        op = BSM(kind='call', S0=S, K=K, T=T, r=r, sigma=sigma)
        delta = op.delta()
        vega = op.vega()
    return delta, vega


In [14]:
# --- Define the longer-maturity replicating option T2 ---

# Original time-to-maturity at t0 (already defined earlier from df_hedge)
T0 = df_hedge.loc[0, "T"]          # e.g. ~45/252 for 45 trading days
# Choose a longer maturity, e.g. +60 trading days. Saadanko oikea data vai pitääkö luoda tälleen
extra_days = 60
extra_T = extra_days / 252.0
T2_0 = T0 + extra_T                # maturity of the replicating call at t0

print(f"Original maturity T0  (years): {T0:.4f} (~{T0*252:.1f} trading days)")
print(f"Replicating maturity T2 (years): {T2_0:.4f} (~{T2_0*252:.1f} trading days)")

# Time grid for original option (T[i]) is already defined:
S = df_hedge["S"].values
C = df_hedge["C"].values
T = df_hedge["T"].values
dates = df_hedge["Date"].values
n = len(S)

# Time-to-maturity of replicating option at each date:
# keep a constant gap (T2_0 - T0)
T_rep = T + (T2_0 - T0)


print(f"Using constant volatility sigma0 = {sigma0:.4f} for both original and replicating calls.")


# --- Simulation of delta–vega hedging ---

def simulate_delta_vega_hedge(
    S, C, T, T_rep, K, r, sigma,
    rebalance_every=1,
    cost_rate=0.0
):
    """
    Delta–vega hedging simulation.

    Portfolio:
      OP: long 1 unit of original call with market price C[i].
      RE: positions alpha_i in stock and eta_i in the replicating call.

    Hedging conditions (local neutrality at each rebalance):
      Delta(Π) = Delta_BS + alpha + eta * Delta_rep = 0
      Vega(Π)  =  Vega_BS        + eta * Vega_rep  = 0

    => eta  = - Vega_BS / Vega_rep
       alpha = -Delta_BS - eta * Delta_rep

    Inputs:
      S, C:       arrays of underlying and original call prices (len n)
      T:          time to maturity of original option at each date
      T_rep:      time to maturity of replicating option at each date
      K, r, sigma: strike, rate, volatility
      rebalance_every: rebalance hedge every k days
      cost_rate:  proportional transaction cost on stock and replicating
                  option trades (e.g. 0.05 for 5%)

    Returns dict with hedging errors and positions.
    """

    S = np.asarray(S)
    C = np.asarray(C)
    T = np.asarray(T)
    T_rep = np.asarray(T_rep)

    n = len(S)

    alpha = np.zeros(n)  # position in underlying
    eta = np.zeros(n)    # position in replicating call

    # For error tracking
    A_no_cost = np.zeros(n - 1)
    A_with_cost = np.zeros(n - 1)

    # Transaction costs per step: we allow costs on stock and replicating option trades
    costs = np.zeros(n - 1)

    # Keep track of old positions to compute traded amounts
    alpha_old = 0.0
    eta_old = 0.0

    # Also track the replicating option price series (model-based)
    C_rep = np.zeros(n)

    for i in range(n):
        # Price of replicating call at time i (model value)
        if T_rep[i] <= 0:
            C_rep[i] = max(S[i] - K, 0.0)
        else:
            op_rep_i = BSM(kind='call', S0=S[i], K=K, T=T_rep[i], r=r, sigma=sigma)
            C_rep[i] = op_rep_i.value()

    # Now walk forward in time between i and i+1
    for i in range(n - 1):

        # Compute hedging ratios at rebalance times
        if i == 0 or (i % rebalance_every == 0):
            # Greeks of original option (to be hedged)
            Delta_BS, Vega_BS = call_greeks(S[i], K, T[i], r, sigma)

            # Greeks of replicating option
            Delta_rep, Vega_rep = call_greeks(S[i], K, T_rep[i], r, sigma)

            # Avoid division by zero if Vega_rep is (numerically) zero
            if abs(Vega_rep) < 1e-12:
                # If no vega in replicating option, fall back to pure delta hedge
                eta_i = 0.0
                alpha_i = -Delta_BS
            else:
                eta_i = - Vega_BS / Vega_rep
                alpha_i = - Delta_BS - eta_i * Delta_rep

            # Transaction costs from changing positions
            traded_stock = abs(alpha_i - alpha_old) * S[i]
            traded_rep = abs(eta_i - eta_old) * C_rep[i]
            costs[i] = cost_rate * (traded_stock + traded_rep)

            alpha_old = alpha_i
            eta_old = eta_i

        # Store current positions
        alpha[i] = alpha_old
        eta[i] = eta_old

        # Changes over [i, i+1]
        dC = C[i + 1] - C[i]           # original option price change
        dS = S[i + 1] - S[i]           # underlying change
        dC_rep = C_rep[i + 1] - C_rep[i]

        # Change in value of replicating portfolio RE
        #   RE value V_RE = alpha * S + eta * C_rep
        #   => ΔV_RE ≈ alpha_i * dS + eta_i * dC_rep
        dV_RE = alpha[i] * dS + eta[i] * dC_rep

        # Hedging error:
        #   A_i = change in OP – change in RE
        A_no_cost[i] = dC - dV_RE

        # Including transaction costs paid at time i
        A_with_cost[i] = A_no_cost[i] - costs[i]

    # Keep last positions for completeness
    alpha[-1] = alpha_old
    eta[-1] = eta_old

    E_no_cost = np.mean(A_no_cost**2)
    E_with_cost = np.mean(A_with_cost**2)

    result = {
        "alpha": alpha,
        "eta": eta,
        "C_rep": C_rep,
        "A_no_cost": A_no_cost,
        "A_with_cost": A_with_cost,
        "E_no_cost": E_no_cost,
        "E_with_cost": E_with_cost,
        "costs": costs,
        "total_cost": float(np.sum(costs)),
    }
    return result

def simulate_delta_vega_hedge_iv(
    S, C, T, T_rep, K, r,
    sigma_init,
    rebalance_every=1,
    cost_rate=0.0
):
    """
    Delta–vega hedging with implied volatility recalibrated at re-hedging dates.

    Inputs
    ------
    S, C, T, T_rep : 1D arrays (len n)
        S[i]     : spot at time i
        C[i]     : market price of original call at time i
        T[i]     : time to maturity (years) of original call at time i
        T_rep[i] : time to maturity (years) of replicating call at time i
    K, r         : strike and risk-free rate
    sigma_init   : initial guess for implied vol at t0
    rebalance_every : rebalance every k steps (e.g. 1 = daily)
    cost_rate    : proportional transaction cost on traded notional
                   (on stock + replicating option)

    Returns
    -------
    dict with alpha, eta, sigma_used, C_rep, A_no_cost, A_with_cost,
    E_no_cost, E_with_cost, costs, total_cost.
    """

    S = np.asarray(S)
    C = np.asarray(C)
    T = np.asarray(T)
    T_rep = np.asarray(T_rep)
    n = len(S)

    alpha = np.zeros(n)         # stock position
    eta = np.zeros(n)           # replicating call position
    sigma_used = np.zeros(n)    # sigma in use at each time
    C_rep = np.zeros(n)         # model price of replicating call

    A_no_cost = np.zeros(n - 1)
    A_with_cost = np.zeros(n - 1)
    costs = np.zeros(n - 1)

    alpha_old = 0.0
    eta_old = 0.0
    sigma_current = sigma_init

    for i in range(n - 1):

        # --- Re-hedge (and recalibrate implied vol) at selected times ---
        if i == 0 or (i % rebalance_every == 0):
            # 1) Recompute implied vol from market price C[i]
            if T[i] > 0 and C[i] > 0 and S[i] > 0:
                try:
                    op_iv = BSM(
                        kind='call',
                        S0=S[i],
                        K=K,
                        T=T[i],
                        r=r,
                        sigma=max(1e-4, sigma_current)
                    )
                    sigma_iv = op_iv.implied_vol(value=C[i])
                    if np.isfinite(sigma_iv) and sigma_iv > 0:
                        sigma_current = sigma_iv
                except Exception:
                    # If implied vol fails, keep previous sigma_current
                    pass

            # 2) Greeks of original call at sigma_current
            Delta_BS, Vega_BS = call_greeks(S[i], K, T[i], r, sigma_current)

            # 3) Greeks of replicating call at sigma_current
            Delta_rep, Vega_rep = call_greeks(S[i], K, T_rep[i], r, sigma_current)

            # 4) Compute hedging ratios alpha, eta
            if abs(Vega_rep) < 1e-12:
                # Fallback: no vega in replicating option -> pure delta hedge
                eta_i = 0.0
                alpha_i = -Delta_BS
            else:
                # Solve:
                #   Vega_BS + eta * Vega_rep = 0      -> eta = -Vega_BS/Vega_rep
                #   Delta_BS + alpha + eta*Delta_rep = 0 -> alpha = -Delta_BS - eta*Delta_rep
                eta_i = - Vega_BS / Vega_rep
                alpha_i = - Delta_BS - eta_i * Delta_rep

            # 5) Transaction costs for changing positions
            #    Need price of replicating call at time i (under sigma_current)
            if T_rep[i] <= 0:
                C_rep_i = max(S[i] - K, 0.0)
            else:
                op_rep_i = BSM(
                    kind='call',
                    S0=S[i],
                    K=K,
                    T=T_rep[i],
                    r=r,
                    sigma=sigma_current
                )
                C_rep_i = op_rep_i.value()

            traded_stock = abs(alpha_i - alpha_old) * S[i]
            traded_rep = abs(eta_i - eta_old) * C_rep_i
            costs[i] = cost_rate * (traded_stock + traded_rep)

            alpha_old = alpha_i
            eta_old = eta_i

        # --- Store current positions and sigma in use ---
        alpha[i] = alpha_old
        eta[i] = eta_old
        sigma_used[i] = sigma_current

        # --- Price replicating call at i and i+1 with sigma_current (piecewise constant vol)---
        if T_rep[i] <= 0:
            C_rep_i = max(S[i] - K, 0.0)
        else:
            C_rep_i = BSM(
                kind='call',
                S0=S[i],
                K=K,
                T=T_rep[i],
                r=r,
                sigma=sigma_current
            ).value()

        if T_rep[i + 1] <= 0:
            C_rep_ip1 = max(S[i + 1] - K, 0.0)
        else:
            C_rep_ip1 = BSM(
                kind='call',
                S0=S[i + 1],
                K=K,
                T=T_rep[i + 1],
                r=r,
                sigma=sigma_current
            ).value()

        # Option and underlying changes over [i, i+1]
        dC = C[i + 1] - C[i]
        dS = S[i + 1] - S[i]
        dC_rep = C_rep_ip1 - C_rep_i

        # Change in replicating portfolio value:
        # V_RE = alpha * S + eta * C_rep
        dV_RE = alpha[i] * dS + eta[i] * dC_rep

        # Hedging errors
        A_no_cost[i] = dC - dV_RE
        A_with_cost[i] = A_no_cost[i] - costs[i]

        # Store replicating prices for reference
        C_rep[i] = C_rep_i
        if i == n - 2:
            C_rep[i + 1] = C_rep_ip1

    # Fill last entries
    alpha[-1] = alpha_old
    eta[-1] = eta_old
    sigma_used[-1] = sigma_current

    E_no_cost = np.mean(A_no_cost**2)
    E_with_cost = np.mean(A_with_cost**2)

    return {
        "alpha": alpha,
        "eta": eta,
        "sigma_used": sigma_used,
        "C_rep": C_rep,
        "A_no_cost": A_no_cost,
        "A_with_cost": A_with_cost,
        "E_no_cost": E_no_cost,
        "E_with_cost": E_with_cost,
        "costs": costs,
        "total_cost": float(np.sum(costs)),
    }



# --- Example usage: compare daily delta–vega hedge to pure delta hedge ---

cost_rate_dv = 0.05    # 5% proportional transaction cost (toy example)
rebalance_every_dv = 1 # daily

res_dv = simulate_delta_vega_hedge(
    S=S,
    C=C,
    T=T,
    T_rep=T_rep,
    K=K,
    r=r,
    sigma=sigma0,
    rebalance_every=rebalance_every_dv,
    cost_rate=cost_rate_dv, 
)

print("=== Delta–vega hedge (daily rebalancing) ===")
print(f"  MSE without costs: {res_dv['E_no_cost']:.6e}")
print(f"  MSE with costs:    {res_dv['E_with_cost']:.6e}")
print(f"  Total transaction cost: {res_dv['total_cost']:.4f}")

# If you still have your delta-only results for rebalance_every=1 and the same cost_rate, you can compare:
if 1 in results:
    print("\n=== Comparison to pure delta hedge (daily) ===")
    print(f"  Delta-only MSE (no cost): {results[1]['E_no_cost']:.6e}")
    print(f"  Delta-only MSE (with cost): {results[1]['E_with_cost']:.6e}")
    print(f"  Delta-only total cost: {results[1]['total_cost']:.4f}")
    
    
cost_rate_dv = 0.05          # 5% transaction costs (toy example)
rebalance_every_dv = 1       # daily

res_dv_iv = simulate_delta_vega_hedge_iv(
    S=S,
    C=C,
    T=T,
    T_rep=T_rep,
    K=K,
    r=r,
    sigma_init=sigma0,
    rebalance_every=rebalance_every_dv,
    cost_rate=cost_rate_dv
)

print("=== Delta–vega hedge with time-varying implied vol (daily) ===")
print(f"  MSE without costs: {res_dv_iv['E_no_cost']:.6e}")
print(f"  MSE with costs:    {res_dv_iv['E_with_cost']:.6e}")
print(f"  Total transaction cost: {res_dv_iv['total_cost']:.4f}")


Original maturity T0  (years): 0.1429 (~36.0 trading days)
Replicating maturity T2 (years): 0.3810 (~96.0 trading days)
Using constant volatility sigma0 = 0.2000 for both original and replicating calls.
=== Delta–vega hedge (daily rebalancing) ===
  MSE without costs: 4.252309e+01
  MSE with costs:    4.968611e+01
  Total transaction cost: 47.0661

=== Comparison to pure delta hedge (daily) ===
  Delta-only MSE (no cost): 1.755061e+01
  Delta-only MSE (with cost): 2.296220e+01
  Delta-only total cost: 38.7684
=== Delta–vega hedge with time-varying implied vol (daily) ===
  MSE without costs: 3.772296e+01
  MSE with costs:    5.200769e+01
  Total transaction cost: 65.7562


In [9]:

def simulate_delta_hedge_portfolio(
    S, C_mat, T, K_vec, weights,
    r, sigma0,
    rebalance_every=1,
    cost_rate=0.0,
    use_implied_vol=False,
    iv_ref_idx=1,
):
    """
    Delta hedging for an option portfolio (e.g. butterfly).

    Parameters
    ----------
    S : 1D array, shape (n,)
        Underlying prices.
    C_mat : 2D array, shape (n, m)
        Matrix of option prices. Column j corresponds to strike K_vec[j].
        Example: columns = [C_low, C_mid, C_high].
    T : 1D array, shape (n,)
        Time-to-maturity in years (same for all strikes at a given i).
    K_vec : 1D array, shape (m,)
        Strikes of the options (e.g. [K1, K2, K3]).
    weights : 1D array, shape (m,)
        Portfolio weights for each option.
        Butterfly example: [1, -2, 1] (long 1 low, short 2 mid, long 1 high).
    r : float
        Risk-free rate.
    sigma0 : float
        Initial volatility (constant if use_implied_vol=False, or initial IV guess).
    rebalance_every : int
        Rehedge every k steps (1 = daily, 2 = every 2 days, ...).
    cost_rate : float
        Proportional transaction cost on underlying trades (e.g. 0.05 = 5%).
    use_implied_vol : bool
        If False: constant sigma0 used for all deltas.
        If True : recalibrate sigma at each rehedge using the reference option column.
    iv_ref_idx : int
        Column index in C_mat and K_vec used to compute implied vol (e.g. mid strike index).

    Returns
    -------
    dict with keys:
        "deltas"               : delta of the *portfolio* at each time
        "A_no_cost"            : step hedging errors without costs
        "A_with_cost"          : step hedging errors with costs
        "E_no_cost"            : MSE without costs
        "E_with_cost"          : MSE with costs
        "costs"                : transaction cost per step
        "total_cost"           : sum of costs
        "cum_error_no_cost"    : cumulative error without costs
        "cum_error_with_cost"  : cumulative error with costs
        "sigma_used"           : volatility used at each time step
    """

    S = np.asarray(S)
    C_mat = np.asarray(C_mat)
    T = np.asarray(T)
    K_vec = np.asarray(K_vec)
    weights = np.asarray(weights)

    n, m = C_mat.shape
    assert len(S) == n and len(T) == n
    assert len(K_vec) == m and len(weights) == m

    # Portfolio prices from market (butterfly, etc.)
    C_port = (C_mat * weights).sum(axis=1)

    # Storage
    deltas = np.zeros(n)          # portfolio delta
    A_no_cost = np.zeros(n - 1)
    A_with_cost = np.zeros(n - 1)
    costs = np.zeros(n - 1)
    sigma_used = np.zeros(n)

    current_delta = 0.0
    sigma_current = sigma0

    for i in range(n - 1):
        # Rehedge at start of [i, i+1]
        if i == 0 or (i % rebalance_every == 0):

            # --- 1) Optionally recalibrate implied vol from reference option ---
            if use_implied_vol and T[i] > 0 and S[i] > 0 and C_mat[i, iv_ref_idx] > 0:
                try:
                    op_iv = BSM(
                        kind="call",
                        S0=S[i],
                        K=K_vec[iv_ref_idx],
                        T=T[i],
                        r=r,
                        sigma=max(1e-4, sigma_current)  # initial guess
                    )
                    sigma_iv = op_iv.implied_vol(value=C_mat[i, iv_ref_idx])
                    if np.isfinite(sigma_iv) and sigma_iv > 0:
                        sigma_current = sigma_iv
                except Exception:
                    # If we fail to find IV, keep previous sigma_current
                    pass

            # --- 2) Compute portfolio delta = sum_j weights_j * Delta_j ---
            if T[i] <= 0:
                # At/after maturity: each call delta is 1{S > K}, so:
                deltas_i = (weights * (S[i] > K_vec).astype(float)).sum()
            else:
                deltas_i = 0.0
                for j in range(m):
                    op_j = BSM(
                        kind="call",
                        S0=S[i],
                        K=K_vec[j],
                        T=T[i],
                        r=r,
                        sigma=sigma_current
                    )
                    deltas_i += weights[j] * op_j.delta()

            new_delta = deltas_i

            # --- 3) Transaction cost from changing underlying position ---
            traded = abs(new_delta - current_delta)
            costs[i] = cost_rate * traded * S[i]

            current_delta = new_delta

        deltas[i] = current_delta
        sigma_used[i] = sigma_current

        # --- 4) Hedging error over [i, i+1] ---
        dC_port = C_port[i + 1] - C_port[i]
        dS = S[i + 1] - S[i]

        A_no_cost[i] = dC_port - current_delta * dS
        A_with_cost[i] = A_no_cost[i] - costs[i]

    # Last entries
    deltas[-1] = current_delta
    sigma_used[-1] = sigma_current

    E_no_cost = np.mean(A_no_cost**2)
    E_with_cost = np.mean(A_with_cost**2)

    cum_error_no_cost = np.cumsum(A_no_cost)
    cum_error_with_cost = np.cumsum(A_with_cost)

    return {
        "deltas": deltas,
        "A_no_cost": A_no_cost,
        "A_with_cost": A_with_cost,
        "E_no_cost": E_no_cost,
        "E_with_cost": E_with_cost,
        "costs": costs,
        "total_cost": float(np.sum(costs)),
        "cum_error_no_cost": cum_error_no_cost,
        "cum_error_with_cost": cum_error_with_cost,
        "sigma_used": sigma_used,
    }


In [16]:
# Pick three strikes 
col_low  = "C460"
col_mid  = "C470"
col_high = "C480"

K_low  = int(col_low[1:])   # 460
K_mid  = int(col_mid[1:])   # 470
K_high = int(col_high[1:])  # 480

K_vec = np.array([K_low, K_mid, K_high])
weights = np.array([1.0, -2.0, 1.0])   # butterfly

# Build the time series
df_bfly = data[["Date", "Underlying", col_low, col_mid, col_high]].dropna().reset_index(drop=True)
S_b = df_bfly["Underlying"].values
C_mat_b = df_bfly[[col_low, col_mid, col_high]].values

# Time to maturity T (years)
n_b = len(df_bfly)
dt = 1.0 / 252.0
T_b = (n_b - 1 - np.arange(n_b)) * dt

# Risk-free rate & initial vol 
r = 0.0


In [17]:
res_bfly_iv = simulate_delta_hedge_portfolio(
    S=S_b,
    C_mat=C_mat_b,
    T=T_b,
    K_vec=K_vec,
    weights=weights,
    r=r,
    sigma0=sigma0,
    rebalance_every=1,
    cost_rate=0.05,
    use_implied_vol=True,   # <-- recompute IV at rehedge points
    iv_ref_idx=1            # use the middle strike as IV reference
)

print("\nButterfly, time-varying implied σ:")
print("  MSE (no cost):", res_bfly_iv["E_no_cost"])
print("  MSE (with cost):", res_bfly_iv["E_with_cost"])
print("  Total trading cost:", res_bfly_iv["total_cost"])


Butterfly, time-varying implied σ:
  MSE (no cost): 142.07664610496786
  MSE (with cost): 140.86170734023523
  Total trading cost: 2.0661983914076996


/opt/software/lib/python3.10/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/software/lib/python3.10/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
